# 05 — Full Pipeline End-to-End Demo

Demonstrates the complete Macro Regime Nowcasting pipeline:

```
FRED API → DataPipeline → DynamicFactorModel → RegimeSwitchingModel
        → Nowcaster → NowcastResult → RegimeAllocator → NarrativeAgent
```

**Prerequisites:** `FRED_API_KEY` (and optionally `OPENAI_API_KEY`) in `.env`


In [ ]:
import sys, os
sys.path.insert(0, os.path.abspath('..'))
from dotenv import load_dotenv
load_dotenv()

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
%matplotlib inline

## Step 1: Fetch Data

In [ ]:
from src.data.fred_client import FREDClient
from src.data.data_pipeline import DataPipeline

client = FREDClient()
pipeline = DataPipeline(fred_client=client, start_date='2000-01-01')
panel = pipeline.run(save_vintage=False)
print(f'Panel: {panel.shape[0]} months × {panel.shape[1]} series')

## Step 2: Run Nowcast

In [ ]:
from src.models.nowcaster import Nowcaster

nowcaster = Nowcaster(pipeline=pipeline, n_factors=4, n_regimes=4)
result = nowcaster.run()
print(nowcaster.get_summary())

## Step 3: Asset Allocation

In [ ]:
from src.allocation.regime_allocator import RegimeAllocator

allocator = RegimeAllocator()
allocation = allocator.get_allocation(result.regime_probabilities)
print('Blended portfolio allocation:')
for asset, weight in sorted(allocation.items(), key=lambda x: -x[1]):
    print(f'  {asset:<15s} {weight:.1%}')

fig = px.pie(names=list(allocation.keys()), values=list(allocation.values()),
             title='Current Blended Allocation')
fig.show()

## Step 4: Narrative Agent (requires OPENAI_API_KEY)

In [ ]:
from src.agent.narrative_agent import NarrativeAgent
from src.agent.fed_scraper import FedScraper

# Optionally fetch recent Fed documents
# scraper = FedScraper()
# fed_docs = scraper.fetch_all_recent(n_each=2)

agent = NarrativeAgent()  # uses OPENAI_API_KEY from env
report = agent.generate(result.to_dict())  # , fed_documents=fed_docs)

print('=== Narrative Summary ===')
print(report.summary)
print()
print('=== Key Drivers ===')
for d in report.key_drivers:
    print(f'  • {d}')
print()
print('=== Risk Flags ===')
for r in report.risk_flags:
    print(f'  ⚠ {r}')